In [3]:
import cv2
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import os
import warnings
warnings.filterwarnings('ignore')


In [3]:
# Define paths to dataset
awake_path = r"C:\Users\Anurag Chauhan\Datasets\drowsy data\test\awake"
sleepy_path = r"C:\Users\Anurag Chauhan\Datasets\drowsy data\test\sleepy"

# Load images and labels
awake = []
sleepy = []

for img in os.listdir(awake_path):
    img_path = os.path.join(awake_path, img)
    img = cv2.imread(img_path)
    img = cv2.resize(img, (224, 224))  # Resize images to uniform size
    awake.append(img)

for img in os.listdir(sleepy_path):
    img_path = os.path.join(sleepy_path, img)
    img = cv2.imread(img_path)
    img = cv2.resize(img, (224, 224))  # Resize images to uniform size
    sleepy.append(img)

# Convert lists to numpy arrays
awake = np.array(awake)
sleepy = np.array(sleepy)

# Define labels
awake_labels = np.zeros(len(awake))
sleepy_labels = np.ones(len(sleepy))

# Combine data and labels
X = np.concatenate((awake, sleepy), axis=0)
y = np.concatenate((awake_labels, sleepy_labels), axis=0)

# Normalize pixel values
X = X / 255.0
 

In [ ]:
# Define model architecture
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(224, 224, 3)),
    MaxPooling2D((2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Flatten(),
    Dense(128, activation='relu'),
    Dense(1, activation='sigmoid')
])

# Compile model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train model
model.fit(X, y, epochs=10, batch_size=32, validation_split=0.2)


In [5]:
# Save the trained model to a file for future use
model.save('driver_drowsiness_model.keras')

# Confirm the model has been saved
import os
print(os.path.exists('driver_drowsiness_model.keras')) 
 

True


In [4]:
import tensorflow as tf
import time
import winsound 


In [ ]:
# Load the saved model
loaded_model = tf.keras.models.load_model('driver_drowsiness_model.keras')

# Load face and eye cascade classifiers
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
eye_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_eye.xml')

# Initialize video capture
cap = cv2.VideoCapture(0)

# Initialize alarm flag and timer
alarm = False
closed_eyes_time = 0
last_closed_eyes_time = time.time()

while True:
    ret, frame = cap.read()
    
    if not ret: 
        break 
    
    # Convert frame to grayscale
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    
    # Detect faces
    faces = face_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5, minSize=(30, 30))
    
    eyes_closed = False
    
    for (x, y, w, h) in faces:
        # Draw rectangle around face
        cv2.rectangle(frame, (x, y), (x+w, y+h), (0, 255, 0), 2)
        
        # Extract ROI for face
        roi_gray = gray[y:y+h, x:x+w]
        roi_color = frame[y:y+h, x:x+w]
        
        # Detect eyes
        eyes = eye_cascade.detectMultiScale(roi_gray)
        
        for (ex, ey, ew, eh) in eyes:
            # Draw rectangle around eyes
            cv2.rectangle(roi_color, (ex, ey), (ex+ew, ey+eh), (255, 0, 0), 2)
            
            # Extract eye region
            eye = roi_color[ey:ey+eh, ex:ex+ew]
            
            # Resize eye to match model input size
            eye = cv2.resize(eye, (224, 224))
            
            # Predict if eye is open or closed using the loaded model
            prediction = loaded_model.predict(np.expand_dims(eye / 255.0, axis=0))
            
            if prediction < 0.5:
                eyes_closed = True
                break
        
        if eyes_closed:
            break
    
    # Update alarm status based on eye state
    if eyes_closed:
        current_time = time.time()
        if last_closed_eyes_time == 0:
            last_closed_eyes_time = current_time
        closed_eyes_time += current_time - last_closed_eyes_time
        last_closed_eyes_time = current_time
        if closed_eyes_time >= 3:
            alarm = True
    else:
        alarm = False
        closed_eyes_time = 0
        last_closed_eyes_time = 0
    
    # Sound alarm if drowsy
    if alarm:
        cv2.putText(frame, "DROWSY", (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)
        # Play alarm sound
        winsound.Beep(2500, 1000)
    
    # Display frame
    cv2.imshow('Drowsiness Detection', frame)
    
    # Exit on key press
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release resources
cap.release()
cv2.destroyAllWindows() 


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 227ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 83ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 81ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 104ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 124ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 118ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 58ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 66ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 102ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 117ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 65ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 52ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━